# Mimic Search Sanitization Tables in Staging (mozdata) 

### In the shared-prod environment, search terms sanitization modifies four tables:

1. It adds lots of rows, one for each of a all search terms deemed sanitary from the 48 hour raw terms table, to the sanitized terms table, which we keep for 30 days
2. It adds a row to a metadata table that records whether a run succeeded, how long it took, why it failed (if it failed), and some aggregate metrics about the terms seen in that run
3. It adds several rows to a languages table for every language seen in the last run, and how many search terms that run saw that the job detected for each language
4. It adds a row to a data validation table that records the proportion of all search terms in the run that displayed various characteristics (capital letters, U.S. 2010 census surnames, English language, etc)

### We want to be able to run and test jobs that write to these tables without messing up the production versions. 

So this notebook, run start to finish, populates tables of the same schema as prod in `mozdata`. You can see the names of the mozdata versions of the tables in the `Args` attributes below.

In [1]:
class Args:
    pass

args = Args()

# For populating metadata table (2)
args.sanitized_term_destination = 'mozdata.search_terms_unsanitized_analysis.prototype_sanitized_data'
args.job_reporting_destination = 'mozdata.search_terms_unsanitized_analysis.prototype_sanitization_job_metadata'
args.unsanitized_term_sample_destination = 'mozdata.search_terms_unsanitized_analysis.prototype_unsanitized_sample_data'

# For populating languages table (3) 
args.job_metadata_origin = 'mozdata.search_terms_unsanitized_analysis.prototype_sanitization_job_metadata'

# For populating data validation table (4)
args.job_reporting_origin = 'mozdata.search_terms_unsanitized_analysis.prototype_sanitization_job_metadata'
args.languages_origin = 'mozdata.search_terms_unsanitized_analysis.prototype_sanitization_job_languages'
args.data_validation_destination = 'mozdata.search_terms_unsanitized_analysis.prototype_data_validation_metrics'

### Step 1: Populate sanitized search terms table from unsanitized search term logs and record sanitization job metadata

In [9]:
from query_sanitization import stream_search_terms, detect_pii, export_search_queries_to_bigquery, record_job_metadata, export_sample_to_bigquery
import numpy
from datetime import datetime, timedelta
import pandas as pd

import collections
import functools
import operator

import asyncio

In [12]:
! python ./sanitation_job.py --sanitized_term_destination=mozdata.search_terms_unsanitized_analysis.prototype_sanitized_data --job_reporting_destination=mozdata.search_terms_unsanitized_analysis.prototype_sanitization_job_metadata --unsanitized_term_sample_destination=mozdata.search_terms_unsanitized_analysis.prototype_unsanitized_sample_data

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],

### Step 2: Populate Languages Table (takes metadata table as input)

In [7]:
from google.cloud import bigquery

In [8]:
client = bigquery.Client()

POPULATE_LANGUAGES_TABLE = f"""
CREATE OR REPLACE TABLE
  `mozdata.search_terms_unsanitized_analysis.prototype_sanitization_job_languages`
AS
SELECT
  started_at AS job_start_time,
  lang.key AS language_code,
  SAFE_CAST(lang.value AS int) AS search_term_count
FROM
  `{args.job_metadata_origin}`
CROSS JOIN
  UNNEST(mozfun.json.js_extract_string_map(approximate_language_proportions_json)) AS lang
"""

query_job = client.query(POPULATE_LANGUAGES_TABLE)

### Step 3: Populate Data Validation Metrics Table (takes metadata table and languages table as input)

In [9]:
from data_validation import calculate_data_validation_metrics, export_data_validation_metrics_to_bigquery

In [10]:
df_i = calculate_data_validation_metrics(args.job_reporting_origin, args.languages_origin)

In [11]:
df_i.finished_at = df_i.finished_at.apply(lambda timestamp: timestamp.strftime("%Y-%m-%d %H:%M:%S"))

In [12]:
export_data_validation_metrics_to_bigquery(df_i, args.data_validation_destination)

[[]]
